In [1]:
# !pip install transformers
# !pip install fugashi
# !pip install unidic
# need transformers, protobuf, fugashi
from transformers import AutoTokenizer, VisionEncoderDecoderModel

tokenizer = AutoTokenizer.from_pretrained("kha-white/manga-ocr-base")
model = VisionEncoderDecoderModel.from_pretrained("kha-white/manga-ocr-base")



C:\Users\hp\Anaconda3\envs\arslan_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\hp\Anaconda3\envs\arslan_env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related

In [3]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToTensor
from tqdm import tqdm
import openpyxl

dirpath = "testocr2/"

for i in os.listdir(dirpath):
    if i.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = dirpath + i
        image = Image.open(image_path)
        resize = Resize((224, 224))
        image = resize(image)
        # Define the transformations
        transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL Image to tensor
            # Add any additional transformations as needed
        ])

        # Apply the transformations
        image_tensor = transform(image)

        
        # Add an extra dimension to simulate batch size of 1
        image_tensor = image_tensor.unsqueeze(0)

        # Pass the image through the model
        generated_ids = model.generate(image_tensor)

        # Decode generated text
        generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        print(i + generated_text)

1.1.jpgえ
1.jpgえ
2.jpgか
3.jpgへ 、
4.jpgで
5.jpgに
6.1.jpgは
6.jpgは
7.jpgへ
8.jpgち
words.jpgw o r d s


In [7]:
def jap_ocr(image_path):
    image = Image.open(image_path)
    resize = Resize((224, 224))
    image = resize(image)
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)
    
    generated_ids = model.generate(image_tensor)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [9]:
import pandas as pd
import glob

# Specify the path to your CSV files
path = "../../pilot data/cl_author"  # Update this to the path of your CSV files
all_files = glob.glob(path + "/*.csv")

# List to hold data from each CSV
li = []

# Loop through all files and read them into a dataframe, then append to the list
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concatenate all dataframes in the list into one dataframe
frame = pd.concat(li, axis=0, ignore_index=True)

# Optionally, you can save the merged dataframe to a new CSV file
frame.to_csv('merged.csv', index=False)

In [12]:
import pandas as pd

# image_folder_name = 'seg_letter'
image_folder_path = '../../pilot data/data/'# dir/dir/image
type_classifier = "merged.csv"

workbook = openpyxl.Workbook()
sheet = workbook.active
transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL Image to tensor
            # Add any additional transformations as needed
        ])
label_map = pd.read_csv(type_classifier, index_col=0).to_dict()['Class']
result = []
for img_dir_name in tqdm(os.listdir(image_folder_path)): 
    for filename in os.listdir(os.path.join(image_folder_path, img_dir_name)): 
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(image_folder_path, img_dir_name, filename)
            # 根据label_map中的标签选择OCR函数
            label = label_map.get(filename)
            if label == 'Kana':
                text = jap_ocr(image_path)
            elif label == 'Kanji':
                text = "this is kanji"
            else:
                text = "No OCR due to unknown label"
            result.append((img_dir_name, filename, text))
    
for (img_dir_name,filename,text) in result:
    
    sheet.append((img_dir_name,filename,text))
        
workbook.save("../pilot_data_ocr.xlsx")
workbook.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [06:22<00:00, 47.87s/it]
